# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [83]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [84]:
clean_city_df = pd.read_csv('./output_data/cities.csv').drop(['Unnamed: 0'], axis=1)
clean_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.6000,73.0833,81.54,76,100,19.19,MV,1659152973
1,Qaanaaq,77.4840,-69.3632,36.57,62,0,4.05,GL,1659152975
2,Hermanus,-34.4187,19.2345,44.80,77,99,9.40,ZA,1659152976
3,Saint Paul Harbor,57.7900,-152.4072,54.91,94,100,8.05,US,1659152978
4,Hobart,-42.8794,147.3294,53.22,58,40,12.66,AU,1659152863
...,...,...,...,...,...,...,...,...,...
556,Ardabil,38.2498,48.2933,54.95,100,100,0.00,IR,1659153637
557,Uglovoye,44.8215,33.6045,75.02,64,0,11.81,UA,1659153828
558,Tecpan,14.7623,-90.9947,51.03,97,36,4.38,GT,1659153829
559,Séguéla,7.9611,-6.6731,69.91,83,99,4.05,CI,1659153830


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [85]:
#configuring gmaps
gmaps.configure(api_key=g_key)

locations = clean_city_df[['Lat', 'Lng']]
weights = clean_city_df['Humidity']

In [86]:
#plotting Heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [103]:
# A max temperature lower than 75 degrees but higher than 70.
# Wind speed less than 5 mph.
# Zero cloudiness.

hotel_df = clean_city_df.loc[((clean_city_df['Max Temp'] > 70) & (clean_city_df['Max Temp'] < 75)) & \
                            (hotel_df['Wind Speed'] < 10) & (hotel_df['Cloudiness'] == 0)]

hotel_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,5.000000,5.000000,5.000000,5.000000,5.0,5.000000,5.000000e+00
mean,30.937740,-20.206340,71.752000,59.400000,0.0,3.102000,1.659153e+09
std,23.115939,55.940063,1.202194,14.774979,0.0,0.644608,1.522094e+02
min,-10.212800,-93.218000,70.320000,36.000000,0.0,2.300000,1.659153e+09
25%,39.050000,-48.360300,71.240000,57.000000,0.0,2.840000,1.659153e+09
50%,40.075800,-27.983300,71.470000,60.000000,0.0,2.930000,1.659153e+09
75%,40.755200,20.138900,72.190000,71.000000,0.0,3.440000,1.659154e+09
max,45.020500,48.391000,73.540000,73.000000,0.0,4.000000,1.659154e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [96]:
hotel_df['Hotel Name'] = ""

target_search = "Hotel"
target_type = "Hotel"
target_radius = 5000

hotel_name = []
# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat}, {lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    try:
        hotel_name.append(response["results"][0]["name"])
    except:
        hotel_name.append('NaN')
        print('hotel not found')
    
hotel_df['Hotel Name'] = hotel_name

hotel not found


In [97]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
153,Gjirokastër,40.0758,20.1389,73.54,36,0,4.00,AL,1659153208,Hotel Argjiro
167,Lagoa,39.0500,-27.9833,72.19,71,0,2.84,PT,1659153229,INATEL Graciosa Hotel
293,Palmas,-10.2128,-48.3603,71.47,60,0,2.30,BR,1659153427,Colorado Palmas Hotel
346,Saint Anthony,45.0205,-93.2180,70.32,73,0,3.44,US,1659153506,NaN
362,Basqal,40.7552,48.3910,71.24,57,0,2.93,AZ,1659153529,BASQAL QONAQ EVI. GUEST HOUSE


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
h_locations = hotel_df[["Lat", "Lng"]]

In [101]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(h_locations, info_box_content=hotel_info)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))